In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from music21 import stream, note, chord, instrument
# from create_generator_model import get_notes, LATENT_DIMENSION

In [ ]:
import random

def create_midi(prediction_output, filename):
    """Convert the output from the prediction to notes and create a MIDI file from the notes."""
    offset = 0
    output_notes = []

    # Define a list of instrument classes to choose from
    instrument_classes = [
    instrument.Accordion,
    instrument.AcousticBass,
    instrument.AcousticGuitar,
    instrument.Agogo,
    instrument.Alto,
    instrument.AltoSaxophone,
    instrument.Bagpipes,
    instrument.Banjo,
    instrument.Baritone,
    instrument.BaritoneSaxophone,
    instrument.Bass,
    instrument.BassClarinet,
    instrument.BassDrum,
    instrument.BassTrombone,
    instrument.Bassoon,
    instrument.BongoDrums,
    instrument.BrassInstrument,
    instrument.Castanets,
    instrument.Celesta,
    instrument.Choir,
    instrument.Clarinet,
    instrument.Clavichord,
    instrument.Conductor,
    instrument.CongaDrum,
    instrument.Contrabass,
    instrument.Contrabassoon,
    instrument.Cowbell,
    instrument.CrashCymbals,
    instrument.Cymbals,
    instrument.Dulcimer,
    instrument.ElectricBass,
    instrument.ElectricGuitar,
    instrument.ElectricOrgan,
    instrument.ElectricPiano,
    instrument.EnglishHorn,
    instrument.FingerCymbals,
    instrument.Flute,
    instrument.FretlessBass,
    instrument.Glockenspiel,
    instrument.Gong,
    instrument.Guitar
]

    # Track the last instrument used to avoid redundant instrument changes
    last_instrument = None

    # Create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item[0]
        # Choose an instrument class for this note or chord
        instr_class = random.choice(instrument_classes)
        print(instr_class)
        instr = instr_class()

        # Add the instrument change only if it is different from the last instrument
        if type(last_instrument) != type(instr):
            output_notes.append(instr)
            last_instrument = instr

        # Pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

In [17]:
def generate_music(generator_model, latent_dim, n_vocab, length=500):
    """ Generate new music using the trained generator model """
    # Create random noise as input to the generator
    noise = np.random.normal(0, 1, (1, latent_dim))
    predictions = generator_model.predict(noise)
    
    # Scale back the predictions to the original range
    pred_notes = [x * (n_vocab / 2) + (n_vocab / 2) for x in predictions[0]]
    
    # Map generated integer indices to note names
    pitchnames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pred_notes_mapped = [int_to_note[int(x)] for x in pred_notes]
    
    return pred_notes_mapped[:length]

In [19]:
from music21 import converter, instrument, note, chord, stream
from pathlib import Path

def get_notes():
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in Path("midi").glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return notes

In [20]:
LATENT_DIMENSION = 1000

In [ ]:
# Load the trained generator model
generator_model = load_model("conditional_music_gan_generator.h5")

# Load the processed notes and get the number of unique pitches
notes = get_notes()
n_vocab = len(set(notes))

# Generate new music sequence
generated_music = generate_music(generator_model, LATENT_DIMENSION, n_vocab)

# Create a MIDI file from the generated music
create_midi(generated_music, 'generated_music')

Parsing midi/instr_6.mid
Parsing midi/instr_30.mid
Parsing midi/instr_33.mid
Parsing midi/instr_51.mid
Parsing midi/instr_27.mid
Parsing midi/instr_36.mid
Parsing midi/instr_10.mid
Parsing midi/instr_23.mid
Parsing midi/instr_8.mid
Parsing midi/instr_22.mid
Parsing midi/instr_15.mid
Parsing midi/instr_50.mid
Parsing midi/instr_38.mid
Parsing midi/instr_18.mid
Parsing midi/instr_24.mid
Parsing midi/instr_45.mid
Parsing midi/instr_34.mid
Parsing midi/instr_11.mid
Parsing midi/instr_44.mid
Parsing midi/instr_17.mid
Parsing midi/instr_43.mid
Parsing midi/instr_16.mid
Parsing midi/instr_39.mid
Parsing midi/instr_7.mid
Parsing midi/instr_48.mid
Parsing midi/instr_5.mid
Parsing midi/instr_2.mid
Parsing midi/instr_31.mid
Parsing midi/instr_14.mid
Parsing midi/instr_35.mid
Parsing midi/instr_25.mid
Parsing midi/instr_46.mid
Parsing midi/instr_47.mid
Parsing midi/instr_32.mid
Parsing midi/instr_26.mid
Parsing midi/instr_13.mid
Parsing midi/instr_42.mid
Parsing midi/instr_3.mid
Parsing midi/instr

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Piano'>
<class 'music21.instrument.Violin'>
<class 'music21.instrument.Bass'>
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Bass'>
<class 'music21.instrument.Violin'>
<class 'music21.instrument.Guitar'>
<class 'music21.instrument.Piano'>
<class 'music21.instrument.Guitar'>
<class 'music21.instrument.Bass'>
<class 'music21.instrument.Guitar'>
<class 'music21.instrument.Violin'>
<class 'music21.instrument.Guitar'>
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Violin'>
<class 'music21.instrument.Bass'>
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Flute'>
<class 'music21.instrument.Guitar'>
<class 'music21.instrument.Piano'>
<class 'music21.instrument.Violin'>
<class 'music21.instrument.Piano'>
<class 'music21.instrument.Violin'>
<class 'music21.instrument.Bass'>
<class 'mus

/tmp/ipykernel_206486/3517239568.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_notes_mapped = [int_to_note[int(x)] for x in pred_notes]


In [1]:
import numpy as np
import random
from music21 import stream, note, chord, instrument
from tensorflow.keras.models import load_model

# Function to load the generator model
def load_generator_model(model_path):
    return load_model(model_path)

# Function to generate music using the model's predictions
def generate_music(generator_model, latent_dim, n_vocab, length=500):
    """
    Generate new music using the trained generator model.
    - generator_model: The trained model used for prediction.
    - latent_dim: The input dimension of the latent space for the generator.
    - n_vocab: The number of unique notes in the dataset.
    - length: The length of the generated music sequence.
    """
    # Create random noise as input to the generator
    noise = np.random.normal(0, 1, (1, latent_dim))
    
    # Predict using the generator model
    predictions = generator_model.predict(noise)
    
    # Scale back the predictions to the original range of notes
    pred_notes = [x * (n_vocab / 2) + (n_vocab / 2) for x in predictions[0]]

    # Map generated integer indices to note names
    pitchnames = sorted(set(item for item in get_notes()))  # get_notes should return all available notes
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pred_notes_mapped = [int_to_note[int(x)] for x in pred_notes]

    return pred_notes_mapped[:length]

# Function to create MIDI from generated music
def create_midi(prediction_output, filename):
    """Convert the output from the prediction to notes and create a MIDI file."""
    offset = 0
    output_notes = []

    # Define a list of instrument classes to choose from
    instrument_classes = [
        instrument.AcousticGuitar, instrument.Piano, instrument.Flute, 
        instrument.Bass, instrument.Violin, instrument.Guitar
    ]

    # Track the last instrument used to avoid redundant instrument changes
    last_instrument = None

    for pattern in prediction_output:
        # Choose an instrument class for this note or chord
        instr_class = random.choice(instrument_classes)
        instr = instr_class()

        # Add the instrument change only if it is different from the last instrument
        if type(last_instrument) != type(instr):
            output_notes.append(instr)
            last_instrument = instr

        # If pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # If pattern is a single note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            output_notes.append(new_note)

        # Increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

# Usage example
model_path = 'conditional_music_gan_generator.h5'
latent_dim = 1000  # Example latent dimension size
n_vocab = len(get_notes())  # Number of unique notes in your dataset

# Load the generator model
generator_model = load_generator_model(model_path)

# Generate music
generated_notes = generate_music(generator_model, latent_dim, n_vocab)

# Create a MIDI file from the generated notes
create_midi(generated_notes, 'generated_music')


2024-11-27 22:54:48.834955: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 22:54:50.265970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


NameError: name 'get_notes' is not defined